# Problem definition

The this notebook we're trying to forecast Nextbike's revenue for the next period of time.

As of day of writing, we have available data for only full 4 months (Jul - Oct 2019), thus it would be hard to precisely forecast revenue for other months. It makes more sense to forcast for next week or even day.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import os
import pandas as pd

# let matplotlib plots be part of Jupyter Notebook
%matplotlib inline

# set up higher resolution for matplotlib plots
%config InlineBackend.figure_format = 'retina'